In [1]:
from pyspark import SparkContext
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import LogisticRegressionWithSGD
from pyspark.ml.classification import LogisticRegression
from pyspark.mllib.feature import HashingTF

In [2]:
# Cargar ficheros ejemplos spam  y no-spam
# Cada línea corresponde a un email
spam = sc.textFile("spam.txt")
no_spam = sc.textFile("no_spam.txt")

In [3]:
# Creo un objeto HashingTF para extraer características a partir de las lineas de email
tf = HashingTF(numFeatures = 100)

# Cada email se divide en palabras y cada palabra se convierte en una feature que nos permitirá
# posteriormente caracterizar si un email es spam o no
spam_features = spam.map(lambda email: tf.transform(email.split(" ")))
no_spam_features = no_spam.map(lambda email: tf.transform(email.split(" ")))

In [4]:
spam_features.collect()

[SparseVector(100, {0: 1.0, 1: 1.0, 4: 2.0, 8: 1.0, 9: 1.0, 11: 1.0, 16: 3.0, 18: 1.0, 20: 1.0, 21: 1.0, 28: 1.0, 29: 1.0, 37: 1.0, 38: 1.0, 42: 1.0, 44: 2.0, 45: 1.0, 48: 1.0, 50: 1.0, 51: 1.0, 72: 1.0, 75: 1.0, 82: 2.0}),
 SparseVector(100, {0: 1.0, 2: 1.0, 21: 1.0, 26: 1.0, 29: 1.0, 37: 1.0, 50: 1.0, 59: 2.0, 76: 1.0, 80: 1.0}),
 SparseVector(100, {1: 1.0, 3: 1.0, 8: 1.0, 12: 1.0, 17: 1.0, 21: 1.0, 25: 1.0, 34: 2.0, 38: 1.0, 44: 1.0, 45: 1.0, 59: 3.0, 71: 1.0, 75: 2.0, 76: 1.0, 82: 1.0, 84: 1.0, 95: 1.0}),
 SparseVector(100, {0: 2.0, 6: 1.0, 8: 2.0, 9: 1.0, 19: 1.0, 21: 1.0, 25: 1.0, 33: 2.0, 34: 1.0, 37: 1.0, 40: 1.0, 55: 1.0, 62: 1.0, 63: 1.0, 71: 1.0, 72: 2.0, 74: 1.0, 76: 1.0, 79: 1.0, 81: 1.0}),
 SparseVector(100, {0: 1.0, 3: 1.0, 8: 1.0, 14: 1.0, 21: 1.0, 24: 1.0, 29: 1.0, 37: 2.0, 42: 1.0, 50: 1.0, 52: 1.0, 62: 1.0, 71: 1.0, 75: 1.0, 76: 2.0, 80: 1.0, 84: 1.0, 91: 1.0, 92: 1.0})]

In [5]:
no_spam_features.collect()

[SparseVector(100, {0: 1.0, 2: 1.0, 4: 1.0, 16: 1.0, 21: 1.0, 25: 1.0, 28: 1.0, 31: 1.0, 34: 2.0, 37: 1.0, 43: 1.0, 57: 2.0, 62: 2.0, 73: 1.0, 82: 1.0, 83: 2.0, 87: 1.0, 92: 1.0, 97: 1.0}),
 SparseVector(100, {0: 1.0, 4: 1.0, 18: 1.0, 20: 1.0, 21: 1.0, 23: 1.0, 34: 1.0, 37: 1.0, 40: 1.0, 42: 1.0, 46: 1.0, 49: 1.0, 58: 1.0, 62: 1.0, 64: 1.0, 72: 1.0, 76: 2.0, 82: 4.0, 95: 1.0, 96: 1.0}),
 SparseVector(100, {0: 1.0, 2: 1.0, 3: 1.0, 4: 1.0, 9: 2.0, 19: 1.0, 21: 1.0, 24: 1.0, 29: 1.0, 34: 1.0, 37: 2.0, 43: 1.0, 45: 1.0, 46: 1.0, 62: 2.0, 65: 1.0, 74: 1.0, 76: 1.0, 77: 1.0, 80: 1.0, 91: 1.0}),
 SparseVector(100, {0: 1.0, 4: 1.0, 11: 1.0, 12: 1.0, 16: 1.0, 17: 1.0, 21: 1.0, 23: 1.0, 34: 1.0, 36: 1.0, 37: 1.0, 56: 1.0, 62: 2.0, 69: 1.0, 72: 1.0, 73: 1.0, 75: 1.0, 79: 1.0, 81: 1.0, 82: 1.0, 85: 2.0, 93: 1.0}),
 SparseVector(100, {0: 2.0, 3: 1.0, 4: 2.0, 6: 1.0, 11: 1.0, 19: 1.0, 21: 1.0, 37: 2.0, 39: 1.0, 43: 1.0, 53: 2.0, 66: 1.0, 72: 2.0, 73: 1.0, 75: 1.0, 77: 1.0, 80: 1.0, 81: 1.0, 82: 3.0}

In [6]:
# Creamos datasets de etiquetas para casos de spam y no spam
spam_examples = spam_features.map(lambda features: LabeledPoint(1, features))
no_spam_examples = no_spam_features.map(lambda features: LabeledPoint(0, features))

# datos de entrenamiento
training_data = spam_examples.union(no_spam_examples)

# Cacheamos los datos para aplicar posteriormente un modelo iterativo
training_data.cache()

UnionRDD[8] at union at NativeMethodAccessorImpl.java:0

In [7]:
#ejemplo de spam se etiquetan con 1
spam_examples.collect()

[LabeledPoint(1.0, (100,[0,1,4,8,9,11,16,18,20,21,28,29,37,38,42,44,45,48,50,51,72,75,82],[1.0,1.0,2.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0])),
 LabeledPoint(1.0, (100,[0,2,21,26,29,37,50,59,76,80],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0])),
 LabeledPoint(1.0, (100,[1,3,8,12,17,21,25,34,38,44,45,59,71,75,76,82,84,95],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,3.0,1.0,2.0,1.0,1.0,1.0,1.0])),
 LabeledPoint(1.0, (100,[0,6,8,9,19,21,25,33,34,37,40,55,62,63,71,72,74,76,79,81],[2.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0])),
 LabeledPoint(1.0, (100,[0,3,8,14,21,24,29,37,42,50,52,62,71,75,76,80,84,91,92],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0]))]

In [8]:
#ejemplo de spam se etiquetan con 0
no_spam_examples.collect()

[LabeledPoint(0.0, (100,[0,2,4,16,21,25,28,31,34,37,43,57,62,73,82,83,87,92,97],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0])),
 LabeledPoint(0.0, (100,[0,4,18,20,21,23,34,37,40,42,46,49,58,62,64,72,76,82,95,96],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,4.0,1.0,1.0])),
 LabeledPoint(0.0, (100,[0,2,3,4,9,19,21,24,29,34,37,43,45,46,62,65,74,76,77,80,91],[1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0])),
 LabeledPoint(0.0, (100,[0,4,11,12,16,17,21,23,34,36,37,56,62,69,72,73,75,79,81,82,85,93],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0])),
 LabeledPoint(0.0, (100,[0,3,4,6,11,19,21,37,39,43,53,66,72,73,75,77,80,81,82],[2.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,3.0])),
 LabeledPoint(0.0, (100,[0,2,3,4,17,27,37,38,44,51,57,67,76,81,84,89,91,92,96],[2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.

In [9]:
# Ejecutamos nuestro modelo de regresión logística sobre los datos de entrenamiento
model = LogisticRegressionWithSGD.train(training_data)

/opt/spark-2.1.0-bin-hadoop2.7/python/pyspark/mllib/classification.py:313: UserWarning: Deprecated in 2.0.0. Use ml.classification.LogisticRegression or LogisticRegressionWithLBFGS.
  "Deprecated in 2.0.0. Use ml.classification.LogisticRegression or "


In [10]:
# Test on a positive example (spam) and a negative one (ham).
# First apply the same HashingTF feature transformation used on the training data.
test_spam_example = tf.transform("GET this product by sending money to account".split(" "))
test_spam_example2 = tf.transform("If you think you have a virus check this".split(" "))
test_no_spam_example = tf.transform("I am studying python because I want be a python developer for web applications".split(" "))

# Now use the learned model to predict spam/ham for new emails.
print "Prediction for spam test example: %g" % model.predict(test_spam_example)
print "Prediction for spam test example: %g" % model.predict(test_spam_example2)
print "Prediction for no spam test example: %g" % model.predict(test_no_spam_example)

Prediction for spam test example: 1
Prediction for spam test example: 1
Prediction for no spam test example: 0
